# Q1

In [2]:
import numpy as np
import wandb
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split

# Load dataset
(X, y), (X_test, y_test) = fashion_mnist.load_data()

# Class labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Initialize wandb for logging
#wandb.init(project="DL_AA1", entity="cs24m037-iit-madras", name="a1_log_images")



# Select one example per class from the raw data
example_indices = [np.where(y == i)[0][0] for i in range(len(class_names))]
example_images = [X[idx] for idx in example_indices]
example_captions = [class_names[y[idx]] for idx in example_indices]

# Log raw sample images to wandb
#wandb.log({"Raw Sample Images": [wandb.Image(img, caption=cap) for img, cap in zip(example_images, example_captions)]})

## Flatten and Noramalize

In [3]:
# Reshape data to flatten images
X = X.reshape(X.shape[0], -1) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Dataset statistics
M, num_features = X_train.shape
Mval = X_val.shape[0]
Mtest = X_test.shape[0]
num_classes = len(np.unique(y_train))

print(f"Training set size: {M}")
print(f"Validation set size: {Mval}")
print(f"Test set size: {Mtest}")
print(f"Number of classes: {num_classes}")
print(f"Features per example: {num_features}")

Training set size: 54000
Validation set size: 6000
Test set size: 10000
Number of classes: 10
Features per example: 784


## Encoding
- one hot enconding

In [4]:
# One-hot encode labels
y_train_one_hot = np.zeros((num_classes, M))
y_train_one_hot[y_train, np.arange(M)] = 1
y_val_one_hot = np.zeros((num_classes, Mval))
y_val_one_hot[y_val, np.arange(Mval)] = 1
y_test_one_hot = np.zeros((num_classes, Mtest))
y_test_one_hot[y_test, np.arange(Mtest)] = 1

# Transpose data for easier manipulation in models
X_train, X_val, X_test, X = X_train.T, X_val.T, X_test.T, X.T

# Define model parameters
input_nodes, output_nodes = num_features, num_classes


# Activation _ Helper __ Fn

In [5]:
class Sigmoid:
    @staticmethod
    def function(x):
        return 1. / (1. + np.exp(-x))

    @staticmethod
    def derivative(x):
        return Sigmoid.function(x) * (1 - Sigmoid.function(x))

class Relu:
    @staticmethod
    def function(x):
        return np.maximum(0, x)

    @staticmethod
    def derivative(x):
        return (x > 0).astype(float)

class Tanh:
    @staticmethod
    def function(x):
        return np.tanh(x)

    @staticmethod
    def derivative(x):
        return 1 - (np.tanh(x) ** 2)

class Softmax:
    @staticmethod
    def function(x):
        exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_x / np.sum(exp_x, axis=0, keepdims=True)

    @staticmethod
    def derivative(x):
        return Softmax.function(x) * (1 - Softmax.function(x))


# Function to initialize weights and biases
def Init_Parameters(layer_dims, init_mode="xavier"):
    np.random.seed(42)
    parameters = {}  # Renamed params to parameters
    prev_updates = {}
    for i in range(1, len(layer_dims)):
        if init_mode == 'random_normal':
            parameters[f"W{i}"] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        elif init_mode == 'random_uniform':
            parameters[f"W{i}"] = np.random.rand(layer_dims[i], layer_dims[i-1]) * 0.01
        elif init_mode == 'xavier':
            parameters[f"W{i}"] = np.random.randn(layer_dims[i], layer_dims[i-1]) * np.sqrt(2 / (layer_dims[i] + layer_dims[i-1]))
        parameters[f"b{i}"] = np.zeros((layer_dims[i], 1))
        prev_updates[f"W{i}"] = np.zeros((layer_dims[i], layer_dims[i-1]))
        prev_updates[f"b{i}"] = np.zeros((layer_dims[i], 1))
    return parameters, prev_updates

# Forward Pass

In [6]:
# Function for forward propagation
def Forward_Propogation(X, parameters, activation_f):
    L = len(parameters) // 2 + 1
    layer_op, pre_activation = [None] * L, [None] * L
    layer_op[0] = X
    for l in range(1, L):
        W, b = parameters[f"W{l}"], parameters[f"b{l}"]
        pre_activation[l] = np.matmul(W, layer_op[l-1]) + b
        if l == L-1:
            layer_op[l] = Softmax.function(pre_activation[l])
        else:
            layer_op[l] = Sigmoid.function(pre_activation[l]) if activation_f == 'sigmoid' else Relu.function(pre_activation[l]) if activation_f == 'relu' else Tanh.function(pre_activation[l])
    return layer_op[-1], layer_op, pre_activation

# Back Prop

In [7]:

# Function for backpropagation
def Back_Propogation(y_hat, y, layer_op, pre_activation, parameters, activation_f, batch_size, loss, lamb):
    L = len(parameters) // 2
    gradients = {}

    gradients[f"dZ{L}"] = (layer_op[L] - y) if loss == 'categorical_crossentropy' else (layer_op[L] - y) * Softmax.derivative(pre_activation[L])

    for l in range(L, 0, -1):
        gradients[f"dW{l}"] = (np.dot(gradients[f"dZ{l}"], layer_op[l-1].T) + lamb * parameters[f"W{l}"]) / batch_size
        gradients[f"db{l}"] = np.sum(gradients[f"dZ{l}"], axis=1, keepdims=True) / batch_size

        if l > 1:
            activation_derivative = Sigmoid.derivative if activation_f == 'sigmoid' else Relu.derivative if activation_f == 'relu' else Tanh.derivative
            gradients[f"dZ{l-1}"] = np.matmul(parameters[f"W{l}"].T, gradients[f"dZ{l}"]) * activation_derivative(pre_activation[l-1])

    return gradients

In [8]:
# import numpy as np

# # Define network architecture
# layer_dims = [784, 128, 64, 10]  # Example: Input layer (784), two hidden layers (128, 64), Output layer (10)
# activation_function = "relu"  # Change to 'sigmoid' or 'tanh' if needed
# batch_size = 32
# lamb = 0.01  # L2 regularization parameter
# loss_function = 'categorical_crossentropy'

# # Initialize parameters
# parameters, _ = Init_Parameters(layer_dims)

# # Generate dummy input and target labels
# X_dummy = np.random.randn(784, batch_size)  # 784 features, batch of 32 samples
# y_dummy = np.eye(10)[:, np.random.choice(10, batch_size)]  # One-hot encoded random labels

# # Forward pass
# y_hat, layer_op, pre_activation = Forward_Propogation(X_dummy, parameters, activation_function)
# print("Forward Propagation Output Shape:", y_hat.shape)  # Should be (10, batch_size)

# # Backward pass
# gradients = Back_Propogation(y_hat, y_dummy, layer_op, pre_activation, parameters, activation_function, batch_size, loss_function, lamb)

# # Check gradients shape
# for key in gradients:
#     print(f"{key} shape: {gradients[key].shape}")


In [9]:
### Optimizer Functions ###

def update_sgd(parameters, gradients, lr=0.01):
    """Standard SGD update."""
    L = len(parameters) // 2
    for l in range(1, L + 1):
        parameters[f"W{l}"] -= lr * gradients[f"dW{l}"]
        parameters[f"b{l}"] -= lr * gradients[f"db{l}"]
    return parameters

def update_momentum(parameters, gradients, prev_updates, lr=0.01, beta1=0.9):
    """Momentum-based gradient descent update."""
    L = len(parameters) // 2
    if "v" not in prev_updates:
        prev_updates["v"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L+1)}
        prev_updates["v"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L+1)})
    for l in range(1, L + 1):
        prev_updates["v"][f"W{l}"] = beta1 * prev_updates["v"][f"W{l}"] + (1 - beta1) * gradients[f"dW{l}"]
        prev_updates["v"][f"b{l}"] = beta1 * prev_updates["v"][f"b{l}"] + (1 - beta1) * gradients[f"db{l}"]
        parameters[f"W{l}"] -= lr * prev_updates["v"][f"W{l}"]
        parameters[f"b{l}"] -= lr * prev_updates["v"][f"b{l}"]
    return parameters, prev_updates

def update_nesterov(parameters, gradients, prev_updates, lr=0.01, beta1=0.9):
    """Nesterov Accelerated Gradient Descent."""
    L = len(parameters) // 2
    if "v" not in prev_updates:
        prev_updates["v"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L+1)}
        prev_updates["v"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L+1)})
    for l in range(1, L + 1):
        v_prev = prev_updates["v"][f"W{l}"]
        prev_updates["v"][f"W{l}"] = beta1 * v_prev + (1 - beta1) * gradients[f"dW{l}"]
        parameters[f"W{l}"] -= lr * (beta1 * v_prev + (1 - beta1) * gradients[f"dW{l}"])
        prev_updates["v"][f"b{l}"] = beta1 * prev_updates["v"][f"b{l}"] + (1 - beta1) * gradients[f"db{l}"]
        parameters[f"b{l}"] -= lr * prev_updates["v"][f"b{l}"]
    return parameters, prev_updates

def update_rmsprop(parameters, gradients, prev_updates, lr=0.001, beta2=0.999, epsilon=1e-8):
    """RMSprop optimizer update."""
    L = len(parameters) // 2
    if "s" not in prev_updates:
        prev_updates["s"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L+1)}
        prev_updates["s"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L+1)})
    for l in range(1, L + 1):
        prev_updates["s"][f"W{l}"] = beta2 * prev_updates["s"][f"W{l}"] + (1 - beta2) * (gradients[f"dW{l}"] ** 2)
        prev_updates["s"][f"b{l}"] = beta2 * prev_updates["s"][f"b{l}"] + (1 - beta2) * (gradients[f"db{l}"] ** 2)
        parameters[f"W{l}"] -= lr * gradients[f"dW{l}"] / (np.sqrt(prev_updates["s"][f"W{l}"]) + epsilon)
        parameters[f"b{l}"] -= lr * gradients[f"db{l}"] / (np.sqrt(prev_updates["s"][f"b{l}"]) + epsilon)
    return parameters, prev_updates

def update_adam(parameters, gradients, prev_updates, t, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    """Adam optimizer update."""
    L = len(parameters) // 2
    if "v" not in prev_updates:
        prev_updates["v"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L+1)}
        prev_updates["v"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L+1)})
    if "s" not in prev_updates:
        prev_updates["s"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L+1)}
        prev_updates["s"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L+1)})
    for l in range(1, L + 1):
        prev_updates["v"][f"W{l}"] = beta1 * prev_updates["v"][f"W{l}"] + (1 - beta1) * gradients[f"dW{l}"]
        prev_updates["s"][f"W{l}"] = beta2 * prev_updates["s"][f"W{l}"] + (1 - beta2) * (gradients[f"dW{l}"] ** 2)
        v_corrected = prev_updates["v"][f"W{l}"] / (1 - beta1 ** t)
        s_corrected = prev_updates["s"][f"W{l}"] / (1 - beta2 ** t)
        parameters[f"W{l}"] -= lr * v_corrected / (np.sqrt(s_corrected) + epsilon)
    return parameters, prev_updates

# Dispatcher function for optimizer selection
def update_parameters(optimizer, parameters, gradients, prev_updates, t=1, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
    if optimizer == "sgd":
        return update_sgd(parameters, gradients, lr), prev_updates
    elif optimizer == "momentum":
        return update_momentum(parameters, gradients, prev_updates, lr, beta1)
    elif optimizer == "nesterov":
        return update_nesterov(parameters, gradients, prev_updates, lr, beta1)
    elif optimizer == "rmsprop":
        return update_rmsprop(parameters, gradients, prev_updates, lr, beta2, epsilon)
    elif optimizer == "adam":
        return update_adam(parameters, gradients, prev_updates, t, lr, beta1, beta2, epsilon)
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer}")

In [10]:
! pip install wandb


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:

from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt


#############################################
# Cost Function
#############################################
def compute_multiclass_loss(Y, Y_hat, batch_size, loss, lamb, parameters):
    """
    Compute the multiclass loss with an optional L2 regularization term.
    """
    if loss == 'categorical_crossentropy':
         cost = -np.sum(Y * np.log(Y_hat + 1e-8)) / batch_size
    elif loss == 'mse':
         cost = np.sum((Y - Y_hat)**2) / (2 * batch_size)

    # L2 regularization term
    reg_sum = 0
    L = len(parameters) // 2
    for l in range(1, L+1):
         reg_sum += np.sum(np.square(parameters[f"W{l}"]))
    cost += (lamb / (2 * batch_size)) * reg_sum
    return cost

#############################################
# Plotting and Evaluation Functions
#############################################
def plot_cost_curve(train_costs, val_costs):
    plt.plot(range(len(train_costs)), train_costs, 'r', label="Training loss")
    plt.plot(range(len(val_costs)), val_costs, 'lime', label="Validation loss")
    plt.title("Training and Validation Loss vs Number of Epochs", size=14)
    plt.xlabel("Epochs", size=14)
    plt.ylabel("Loss", size=14)
    plt.grid()
    plt.legend()
    plt.show()

def NN_predict(X, parameters, activation_f):
    output, _, _ = Forward_Propogation(X, parameters, activation_f)
    predictions = np.argmax(output, axis=0)
    return predictions

def NN_evaluate(X_train, y_train, X_test, y_test, parameters, activation_f):
    train_preds = NN_predict(X_train, parameters, activation_f)
    test_preds = NN_predict(X_test, parameters, activation_f)
    print("Training Accuracy: {:.3f}%".format(accuracy_score(y_train, train_preds) * 100))
    print("Test Accuracy: {:.3f}%".format(accuracy_score(y_test, test_preds) * 100))
    print("Classification Report:\n", classification_report(y_test, test_preds))
    return train_preds, test_preds

#############################################
# Hyperparameter Tuning using W&B Sweeps (NN_fit)
#############################################
def NN_fit():
    """
    This function is used for hyperparameter tuning via wandb sweeps.
    It assumes that the following are defined in your Q1–Q3 code:
      - Global variables: X_train, X_val, y_train, y_val, y_train_one_hot, y_val_one_hot, num_features, num_classes
      - Functions: Init_Parameters, Forward_Propogation, Back_Propogation, update_sgd, update_momentum,
                   update_nesterov, update_rmsprop, update_adam, and the dispatcher update_parameters.
    """
    # Default hyperparameter values
    config_defaults = {
        'epochs': 10,
        'batch_size': 64,
        'learning_rate': 1e-3,
        'activation_f': 'relu',
        'optimizer': 'adam',  # Options: "sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"
        'init_mode': 'xavier',  # Options: "xavier", "random_normal", "random_uniform"
        'L2_lamb': 0,
        'num_neurons': 64,
        'num_hidden': 3
    }

    wandb.init(config=config_defaults)
    config = wandb.config

    # Extract hyperparameters from wandb config
    epochs        = config.epochs
    batch_size    = config.batch_size
    learning_rate = config.learning_rate
    activation_f  = config.activation_f
    optimizer     = config.optimizer
    init_mode     = config.init_mode
    L2_lamb       = config.L2_lamb
    num_neurons   = config.num_neurons
    num_hidden    = config.num_hidden

    # Construct a meaningful run name for logging
    run_name = "lr_{}_ac_{}_in_{}_op_{}_bs_{}_L2_{}_ep_{}_nn_{}_nh_{}".format(
        learning_rate, activation_f, init_mode, optimizer, batch_size, L2_lamb, epochs, num_neurons, num_hidden)
    print(run_name)

    # Assumed global variables from preprocessing:
    # X_train, X_val, y_train, y_val, y_train_one_hot, y_val_one_hot, num_features, num_classes
    global X_train, X_val, y_train, y_val, y_train_one_hot, y_val_one_hot, num_features, num_classes
    M    = X_train.shape[1]  # Number of training examples
    Mval = X_val.shape[1]    # Number of validation examples

    # Define the neural network architecture: input layer, hidden layers, and output layer
    layer_dims = [num_features] + [num_neurons] * num_hidden + [num_classes]

    # Initialize parameters and previous updates (for momentum-based optimizers)
    parameters, prev_updates = Init_Parameters(layer_dims, init_mode)

    # For optimizers that use Adam (or Nadam), maintain a timestep
    t = 1
    # For lookahead (used in Nesterov and Nadam), create a copy of parameters
    params_look_ahead = parameters.copy()

    # Lists to store cost for each epoch
    epoch_costs = []
    val_costs   = []

    loss = 'mse'
    beta = 0.9  # Momentum parameter

    # Training loop over epochs
    for epoch in range(epochs):
        for i in range(0, M, batch_size):
            current_batch_size = batch_size
            if i + batch_size > M:
                current_batch_size = M - i

            # Select current mini-batch
            X_batch = X_train[:, i:i+batch_size]
            Y_batch = y_train_one_hot[:, i:i+batch_size]

            # Choose update method based on the optimizer
            if optimizer == "nesterov":
                L_layers = len(parameters) // 2
                # Compute lookahead parameters
                for l in range(1, L_layers+1):
                    # If momentum has not been initialized, default to zeros
                    if "v" not in prev_updates:
                        prev_updates["v"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L_layers+1)}
                        prev_updates["v"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L_layers+1)})
                    params_look_ahead[f"W{l}"] = parameters[f"W{l}"] - beta * prev_updates["v"][f"W{l}"]
                    params_look_ahead[f"b{l}"] = parameters[f"b{l}"] - beta * prev_updates["v"][f"b{l}"]

                # Forward propagation with lookahead parameters
                output, layer_op, pre_act = Forward_Propogation(X_batch, params_look_ahead, activation_f)
                gradients = Back_Propogation(output, Y_batch, layer_op, pre_act, params_look_ahead, activation_f, current_batch_size, loss, L2_lamb)
                parameters, prev_updates = update_nesterov(parameters, gradients, prev_updates, lr=learning_rate, beta1=beta)

            elif optimizer == "nadam":
                L_layers = len(parameters) // 2
                # Compute lookahead parameters similar to Nesterov
                for l in range(1, L_layers+1):
                    if "v" not in prev_updates:
                        prev_updates["v"] = {f"W{l}": np.zeros_like(parameters[f"W{l}"]) for l in range(1, L_layers+1)}
                        prev_updates["v"].update({f"b{l}": np.zeros_like(parameters[f"b{l}"]) for l in range(1, L_layers+1)})
                    params_look_ahead[f"W{l}"] = parameters[f"W{l}"] - beta * prev_updates["v"][f"W{l}"]
                    params_look_ahead[f"b{l}"] = parameters[f"b{l}"] - beta * prev_updates["v"][f"b{l}"]

                # Forward propagation with lookahead parameters
                output, layer_op, pre_act = Forward_Propogation(X_batch, params_look_ahead, activation_f)
                gradients = Back_Propogation(output, Y_batch, layer_op, pre_act, params_look_ahead, activation_f, current_batch_size, loss, L2_lamb)
                parameters, prev_updates = update_adam(parameters, gradients, prev_updates, t, lr=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8)
                t += 1

            else:
                # For optimizers: sgd, momentum, rmsprop, and adam (without lookahead)
                output, layer_op, pre_act = Forward_Propogation(X_batch, parameters, activation_f)
                gradients = Back_Propogation(output, Y_batch, layer_op, pre_act, parameters, activation_f, current_batch_size, loss, L2_lamb)
                parameters, prev_updates = update_parameters(optimizer, parameters, gradients, prev_updates, t, lr=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8)
                if optimizer == "adam":
                    t += 1

        # Compute training cost over the full training set
        train_output, _, _ = Forward_Propogation(X_train, parameters, activation_f)
        epoch_cost = compute_multiclass_loss(y_train_one_hot, train_output, M, loss, L2_lamb, parameters)
        epoch_costs.append(epoch_cost)

        # Compute validation cost
        val_output, _, _ = Forward_Propogation(X_val, parameters, activation_f)
        val_cost = compute_multiclass_loss(y_val_one_hot, val_output, Mval, loss, L2_lamb, parameters)
        val_costs.append(val_cost)

        # Compute accuracies
        train_preds = NN_predict(X_train, parameters, activation_f)
        val_preds   = NN_predict(X_val, parameters, activation_f)
        train_acc = accuracy_score(y_train, train_preds)
        val_acc   = accuracy_score(y_val, val_preds)

        # Log metrics to wandb for the current epoch
        wandb.log({
            "epoch": epoch+1,
            "training_loss": epoch_cost,
            "validation_loss": val_cost,
            "training_accuracy": train_acc,
            "validation_accuracy": val_acc
        })

        print(f"Epoch {epoch+1}: Training Loss = {epoch_cost:.4f}, Validation Loss = {val_cost:.4f}, Training Acc = {train_acc:.4f}, Validation Acc = {val_acc:.4f}")

    # Plot cost curves after training is complete
    #plot_cost_curve(epoch_costs, val_costs)

    # Set a meaningful run name in wandb and finish the run
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()

    return parameters, epoch_costs

#############################################
# W&B Sweep Configuration & Execution
#############################################
sweep_config = {
  "name": "CS6910 Assignment 1 - MSE -08-03-2025-0930 hrs",
  "metric": {
      "name": "validation_accuracy",
      "goal": "maximize"
  },
  "method": "random",
  "parameters": {
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        "activation_f": {
            "values": ["sigmoid", "relu", "tanh"]
        },
        "init_mode": {
            "values": ["xavier", "random_uniform"]
        },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
        },
        "batch_size": {
            "values": [16, 32, 64]
        },
        "epochs": {
            "values": [5, 10]
        },
        "L2_lamb": {
            "values": [0, 0.0005, 0.5]
        },
        "num_neurons": {
            "values": [32, 64, 128]
        },
        "num_hidden": {
            "values": [3, 4, 5]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, entity="cs24m037-iit-madras", project="DL_A1")
wandb.agent(sweep_id, NN_fit, count=120)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: zbxy6yvx
Sweep URL: https://wandb.ai/cs24m037-iit-madras/DL_A1/sweeps/zbxy6yvx


wandb: Agent Starting Run: fuy6bog6 with config:
wandb: 	L2_lamb: 0.5
wandb: 	activation_f: relu
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	init_mode: random_uniform
wandb: 	learning_rate: 0.0001
wandb: 	num_hidden: 3
wandb: 	num_neurons: 32
wandb: 	optimizer: nadam
wandb: Currently logged in as: cs24m037 (cs24m037-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


lr_0.0001_ac_relu_in_random_uniform_op_nadam_bs_16_L2_0.5_ep_10_nn_32_nh_3
Epoch 1: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 2: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 3: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 4: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 5: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 6: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 7: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 8: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1001, Validation Acc = 0.0995
Epoch 9: Training Loss = 0.4500, Validation Loss = 0.4500, Training Acc = 0.1

wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


...

lr_0.001_ac_relu_in_random_uniform_op_nadam_bs_32_L2_0_ep_5_nn_32_nh_5
Epoch 1: Training Loss = 0.4135, Validation Loss = 0.4129, Training Acc = 0.0999, Validation Acc = 0.1018
Epoch 2: Training Loss = 0.4082, Validation Loss = 0.4076, Training Acc = 0.0999, Validation Acc = 0.1018
Epoch 3: Training Loss = 0.4057, Validation Loss = 0.4049, Training Acc = 0.0999, Validation Acc = 0.1018
Epoch 4: Training Loss = 0.4040, Validation Loss = 0.4035, Training Acc = 0.0998, Validation Acc = 0.1020
Epoch 5: Training Loss = 0.4014, Validation Loss = 0.4010, Training Acc = 0.1001, Validation Acc = 0.1020


epoch,▁▃▅▆█
training_accuracy,▂▃▄▁█
training_loss,█▅▃▃▁
validation_accuracy,▁▁▁██
validation_loss,█▅▃▂▁
epoch,5
training_accuracy,0.10009
training_loss,0.40138
validation_accuracy,0.102
validation_loss,0.40105


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: b8pb96oh with config:
wandb: 	L2_lamb: 0.0005
wandb: 	activation_f: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	init_mode: xavier
wandb: 	learning_rate: 0.0001
wandb: 	num_hidden: 5
wandb: 	num_neurons: 32
wandb: 	optimizer: nadam


lr_0.0001_ac_sigmoid_in_xavier_op_nadam_bs_32_L2_0.0005_ep_10_nn_32_nh_5
Epoch 1: Training Loss = 0.4327, Validation Loss = 0.4332, Training Acc = 0.2010, Validation Acc = 0.1895
Epoch 2: Training Loss = 0.4058, Validation Loss = 0.4062, Training Acc = 0.1973, Validation Acc = 0.1987
Epoch 3: Training Loss = 0.4031, Validation Loss = 0.4036, Training Acc = 0.1913, Validation Acc = 0.1950
Epoch 4: Training Loss = 0.4020, Validation Loss = 0.4026, Training Acc = 0.2533, Validation Acc = 0.2522
Epoch 5: Training Loss = 0.3824, Validation Loss = 0.3838, Training Acc = 0.3135, Validation Acc = 0.3020
Epoch 6: Training Loss = 0.3637, Validation Loss = 0.3653, Training Acc = 0.3884, Validation Acc = 0.3810
Epoch 7: Training Loss = 0.3479, Validation Loss = 0.3494, Training Acc = 0.3890, Validation Acc = 0.3797
Epoch 8: Training Loss = 0.3344, Validation Loss = 0.3359, Training Acc = 0.4279, Validation Acc = 0.4185
Epoch 9: Training Loss = 0.3234, Validation Loss = 0.3252, Training Acc = 0.485

epoch,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▁▁▂▃▅▅▆▇█
training_loss,█▇▆▆▅▄▃▃▂▁
validation_accuracy,▁▁▁▂▃▅▅▆▇█
validation_loss,█▇▆▆▅▄▃▃▂▁
epoch,10
training_accuracy,0.54459
training_loss,0.30416
validation_accuracy,0.54033
validation_loss,0.30632
